In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Load your dataset
# Assuming you have a dataset with columns 'text' and 'category'
# Replace 'your_dataset.csv' with the actual file path or URL
dataset = pd.read_csv('/content/Train.csv')

In [ ]:
dataset.head()

,id,content,category
0,SW0,SERIKALI imesema haitakuwa tayari kuona amani...,Kitaifa
1,SW1,"Mkuu wa Mkoa wa Tabora, Aggrey Mwanri amesiti...",Biashara
2,SW10,SERIKALI imetoa miezi sita kwa taasisi zote z...,Kitaifa
3,SW100,KAMPUNI ya mchezo wa kubahatisha ya M-bet ime...,michezo
4,SW1000,WATANZANIA wamekumbushwa kusherehekea sikukuu...,Kitaifa


In [ ]:
# Define hyperparameters
max_sequence_length = 100  # Adjust as needed
embedding_dim = 200  # Adjust based on your requirements
num_epochs = 10  # Adjust as needed
max_words = 10000  # Adjust as needed

# Preprocess the text data
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(dataset['content'])
sequences = tokenizer.texts_to_sequences(dataset['content'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

In [ ]:
# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(dataset['category'])

In [ ]:
# Combine text data and labels into a DataFrame
train_data = pd.DataFrame(data={'content': padded_sequences.tolist(), 'category': labels})

In [ ]:
# Define the K-fold cross-validation strategy (Stratified K-fold for classification)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Function to build and compile the model
def build_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))
    model.add(LSTM(100))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Create a KerasClassifier with the build_model function
keras_classifier = KerasClassifier(build_fn=build_model, epochs=num_epochs, verbose=0)

In [ ]:
# Use cross_val_score with KerasClassifier
cv_scores = cross_val_score(keras_classifier, np.array(train_data['content'].tolist()), train_data['category'], cv=kfold, scoring='accuracy')

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras

In [ ]:
# Print the cross-validated accuracy scores
print("Cross-validated Accuracy Scores:", cv_scores)
print("Mean Accuracy: {:.2f}%".format(cv_scores.mean() * 100))

Cross-validated Accuracy Scores: [0.77206596 0.80194175 0.82427184 0.76213592 0.78252427]
Mean Accuracy: 78.86%


In [ ]:
# Train the model on the full dataset
model = build_model()
model.fit(np.array(train_data['content'].tolist()), train_data['category'], epochs=num_epochs)

Epoch 1/10
161/161 [==============================] - 30s 168ms/step - loss: 0.9247 - accuracy: 0.5826
Epoch 2/10
161/161 [==============================] - 27s 167ms/step - loss: 0.4907 - accuracy: 0.8218
Epoch 3/10
161/161 [==============================] - 27s 168ms/step - loss: 0.3297 - accuracy: 0.8880
Epoch 4/10
161/161 [==============================] - 28s 175ms/step - loss: 0.2005 - accuracy: 0.9377
Epoch 5/10
161/161 [==============================] - 27s 168ms/step - loss: 0.1378 - accuracy: 0.9596
Epoch 6/10
161/161 [==============================] - 27s 170ms/step - loss: 0.0946 - accuracy: 0.9709
Epoch 7/10
161/161 [==============================] - 28s 176ms/step - loss: 0.0629 - accuracy: 0.9829
Epoch 8/10
161/161 [==============================] - 27s 168ms/step - loss: 0.0416 - accuracy: 0.9887
Epoch 9/10
161/161 [==============================] - 27s 169ms/step - loss: 0.0340 - accuracy: 0.9913
Epoch 10/10
161/161 [==============================] - 28s 173ms/step - l

In [ ]:
# Save the trained model
save_model(model, 'cross_validated_model.h5')

<ipython-input-200-64b1f04ea756>:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model, 'cross_validated_model.h5')


In [ ]:
# Load the model
loaded_model = load_model('cross_validated_model.h5')

In [ ]:
# Assuming you have a new dataset with the same structure (column 'text')
new_dataset = pd.read_csv('/content/Test.csv')  # Replace with your actual file path or URL

In [ ]:
# Preprocess the new data
new_sequences = tokenizer.texts_to_sequences(new_dataset['content'])
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

In [ ]:
# Make predictions
predictions = loaded_model.predict(new_padded_sequences)

33/33 [==============================] - 2s 43ms/step


In [ ]:
# Decode the predictions back to class labels
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

In [ ]:
# Add the predicted labels to the new dataset
new_dataset['predicted_category'] = predicted_labels

In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = loaded_model.evaluate(X_test, y_test)

33/33 [==============================] - 2s 42ms/step - loss: 0.0343 - accuracy: 0.9874


In [ ]:
# Print the evaluation results
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

Test Loss: 0.034335847944021225
Test Accuracy: 0.9873908758163452


In [ ]:
new_dataset.head()

,swahili_id,content,predicted_category
0,ae3baa6c34aa523fd2aa4de3c89448efff922311,Rais John Magufuli amemuagiza Msajili wa Hazi...,Kitaifa
1,c4ee26a3ade8064a2ec494996e836900fd32dd8e,TAHARUKI imezuka katika mkutano wa Naibu Wazi...,Kitaifa
2,58aee3aa1d94554ff57e6a053dbd60658e4890ff,"KOCHA wa Azam FC ya Dar es Salaam, Idd Cheche...",michezo
3,00579c2307b5c11003d21c40c3ecff5e922c3fd8,THAMANI ya mauzo ya bidhaa za Afrika Masharik...,Kitaifa
4,c83e9738ae5d1790ee85b99863deb734e7614c52,"WAZIRI wa Nchi, Ofi si ya Makamu wa Rais, Muu...",Kitaifa


In [ ]:
# View the new dataset with predicted labels
print(new_dataset[['content', 'predicted_category']])

                                                content predicted_category
0      Rais John Magufuli amemuagiza Msajili wa Hazi...            Kitaifa
1      TAHARUKI imezuka katika mkutano wa Naibu Wazi...            Kitaifa
2      KOCHA wa Azam FC ya Dar es Salaam, Idd Cheche...            michezo
3      THAMANI ya mauzo ya bidhaa za Afrika Masharik...            Kitaifa
4      WAZIRI wa Nchi, Ofi si ya Makamu wa Rais, Muu...            Kitaifa
...                                                 ...                ...
1025   MKURUGENZI wa Jiji la Arusha, Dk Maulid Maden...            Kitaifa
1026   Hayo yalisemwa na Katibu Mkuu Wizara ya Afrik...           Biashara
1027   Alitoa kauli hiyo juzi wakati akizungumza na ...           Biashara
1028   SIMBA imetinga raundi ya nne ya 32 bora ya Ko...            michezo
1029   KOCHA wa Azam FC, Mholanzi Hans van der Pluij...            michezo

[1030 rows x 2 columns]


In [ ]:
# Add one-hot encoding for predicted classes
one_hot_encoded = pd.get_dummies(new_dataset['predicted_category'])

In [ ]:
# Concatenate the entry ID and one-hot encoded classes
submission_df = pd.concat([new_dataset['swahili_id'], one_hot_encoded], axis=1)

In [ ]:
# Save the submission dataframe to a CSV file
submission_df.to_csv('submission.csv', index=False)

In [ ]:
view_csv = pd.read_csv('/content/submission.csv')

In [ ]:
view_csv.head()

,swahili_id,Biashara,Burudani,Kimataifa,Kitaifa,michezo
0,ae3baa6c34aa523fd2aa4de3c89448efff922311,0,0,0,1,0
1,c4ee26a3ade8064a2ec494996e836900fd32dd8e,0,0,0,1,0
2,58aee3aa1d94554ff57e6a053dbd60658e4890ff,0,0,0,0,1
3,00579c2307b5c11003d21c40c3ecff5e922c3fd8,0,0,0,1,0
4,c83e9738ae5d1790ee85b99863deb734e7614c52,0,0,0,1,0
